# Clustering

## Setup

In [ ]:
%conda install -c conda-forge numpy pandas matplotlib seaborn scipy

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from scipy.spatial.distance import cdist
from scipy.stats import multivariate_normal
from sklearn.metrics import silhouette_score
import time

In [2]:
data_pca = np.load('data/features_pca.npz')
features_pca = data_pca['features']
movieIds_pca = data_pca['movieId']

data_svd = np.load('data/features_svd.npz')
features_svd = data_svd['features']
movieIds_svd = data_svd['movieId']

data_lda = np.load('data/features_lda.npz')
features_lda = data_lda['features']
movieIds_lda = data_lda['movieId']

movies = pd.read_csv('data/train_complete.csv')
movies = pd.concat([movies, pd.read_csv('data/test_complete.csv')], ignore_index=True)

## Selected Algorithms

1. K-means (Partitioning method)
2. Gaussian Mixture Model - GMM (Distriburion-based method)

### K-means

- Industry standard for image clustering
- Fast and scalable for large datasers
- Works well when clusters are spherical and similar in size
- Easy to interpret: each movie belongs to exactly one cluster
- Suitable for poster features where visual styles form distinct groups

In [3]:
class KMeans:
    def __init__(self, n_clusters=10, max_iters=100, tol=1e-4, random_state=42):
        self.n_clusters = n_clusters
        self.max_iters = max_iters
        self.tol = tol
        self.random_state = random_state
        np.random.seed(self.random_state)
        
    def fit(self, X):
        n_samples, n_features = X.shape
        
        self.centroids = self._kmeans_plus_plus(X)
        self.labels_ = np.zeros(n_samples, dtype=int)
        self.inertia_ = 0.0
        
        for iteration in range(self.max_iters):
            distances = cdist(X, self.centroids, metric='euclidean')
            self.labels_ = np.argmin(distances, axis=1)
            
            new_centroids = np.array([X[self.labels_ == k].mean(axis=0) 
                                     for k in range(self.n_clusters)])
            
            if np.allclose(self.centroids, new_centroids, atol=self.tol):
                break
                
            self.centroids = new_centroids
        
        self.inertia_ = np.sum((X - self.centroids[self.labels_])**2)
        
        return self
    
    def _kmeans_plus_plus(self, X):
        n_samples = X.shape[0]
        centroids = []
        
        first_idx = np.random.randint(n_samples)
        centroids.append(X[first_idx])
        
        for _ in range(1, self.n_clusters):
            distances = cdist(X, np.array(centroids), metric='euclidean')
            min_distances = np.min(distances, axis=1)
            probabilities = min_distances ** 2
            probabilities /= probabilities.sum()
            
            next_idx = np.random.choice(n_samples, p=probabilities)
            centroids.append(X[next_idx])
        
        return np.array(centroids)
    
    def predict(self, X):
        distances = cdist(X, self.centroids, metric='euclidean')
        return np.argmin(distances, axis=1)
    
    def fit_predict(self, X):
        self.fit(X)
        return self.labels_

### GMM

- Probabilistic assignments: captures uncertainty in cluster memebership
- Flexible cluster shapes: can model elliptical clusters
- Better for overlapping styles: a poster can have mixed characteristics
- Natural for movie posters: genres often blend (action-comedy, scifi-fi-drama, etc)
- Provides probability scores useful for recommendations

In [4]:
class GaussianMixtureModel:
    def __init__(self, n_components=10, max_iters=100, tol=1e-4, random_state=42):
        self.n_components = n_components
        self.max_iters = max_iters
        self.tol = tol
        self.random_state = random_state
        np.random.seed(self.random_state)
        
    def fit(self, X):
        n_samples, n_features = X.shape
        
        self.weights_ = np.ones(self.n_components) / self.n_components
        indices = np.random.choice(n_samples, self.n_components, replace=False)
        self.means_ = X[indices].copy()
        self.covariances_ = np.array([np.eye(n_features) for _ in range(self.n_components)])
        self.labels_ = np.zeros(n_samples, dtype=int)
        
        log_likelihood_old = 0
        
        for iteration in range(self.max_iters):
            responsibilities = self._e_step(X)
            self._m_step(X, responsibilities)
            log_likelihood = self._compute_log_likelihood(X)
            
            if abs(log_likelihood - log_likelihood_old) < self.tol:
                break
                
            log_likelihood_old = log_likelihood
        
        self.labels_ = self.predict(X)
        
        return self
    
    def _e_step(self, X):
        n_samples = X.shape[0]
        responsibilities = np.zeros((n_samples, self.n_components))
        
        for k in range(self.n_components):
            try:
                responsibilities[:, k] = self.weights_[k] * multivariate_normal.pdf(
                    X, mean=self.means_[k], cov=self.covariances_[k], allow_singular=True
                )
            except:
                responsibilities[:, k] = 1e-10
        
        responsibilities_sum = responsibilities.sum(axis=1, keepdims=True)
        responsibilities /= (responsibilities_sum + 1e-10)
        
        return responsibilities
    
    def _m_step(self, X, responsibilities):
        n_samples, n_features = X.shape
        
        Nk = responsibilities.sum(axis=0)
        self.weights_ = Nk / n_samples
        
        self.means_ = (responsibilities.T @ X) / Nk[:, np.newaxis]
        
        for k in range(self.n_components):
            diff = X - self.means_[k]
            weighted_diff = responsibilities[:, k][:, np.newaxis] * diff
            self.covariances_[k] = (weighted_diff.T @ diff) / Nk[k]
            
            self.covariances_[k] += np.eye(n_features) * 1e-6
    
    def _compute_log_likelihood(self, X):
        n_samples = X.shape[0]
        log_likelihood = 0
        
        for k in range(self.n_components):
            try:
                log_likelihood += np.sum(
                    np.log(self.weights_[k] * multivariate_normal.pdf(
                        X, mean=self.means_[k], cov=self.covariances_[k], allow_singular=True
                    ) + 1e-10)
                )
            except:
                pass
        
        return log_likelihood / n_samples
    
    def predict(self, X):
        responsibilities = self._e_step(X)
        return np.argmax(responsibilities, axis=1)
    
    def fit_predict(self, X):
        self.fit(X)
        return self.labels_
    
    def predict_proba(self, X):
        return self._e_step(X)

## Optimal Number of Clusters

### Hint from number of genres

In [5]:
all_genres = set()

for genre_str in movies['genres'].dropna():
    all_genres.update(genre_str.split('|'))

n_genres = len(all_genres)

print(f"Number of unique genres in dataset: {n_genres}")
print(f"Genres: {sorted(all_genres)}")

Number of unique genres in dataset: 19
Genres: ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [ ]:
n_range = range(max(n_genres // 2, 1), n_genres, 1)

reduction_methods = {
    'PCA': (features_pca, movieIds_pca),
    'SVD': (features_svd, movieIds_svd),
    'LDA': (features_lda, movieIds_lda)
}

avg_silhouettes = []

for n in n_range:
    print(f"Evaluating for k={n}")
    k_silhouettes = []

    for method, (features, movieIds) in reduction_methods.items():
        kmeans = KMeans(n_clusters=n, max_iters=50)
        labels = kmeans.fit_predict(features)
        sil = silhouette_score(features, labels)
        k_silhouettes.append(sil)
        print(f"\t{method}: {sil:.4f}")
    
    avg = np.mean(k_silhouettes)
    avg_silhouettes.append(avg)
    print(f"\tAvg: {avg:.4f}")

optimal_n_kmeans = list(n_range)[np.argmax(avg_silhouettes)]
print(f"\nOptimal number of clusters for K-means based on average silhouette score: {optimal_n_kmeans}")

Evaluating for k=9
	PCA: 0.0088
	SVD: 0.0163
	LDA: 0.0743
	Avg: 0.0332
Evaluating for k=10
	PCA: 0.0093
	SVD: 0.0142
	LDA: 0.0816
	Avg: 0.0351
Evaluating for k=11
	PCA: 0.0124
	SVD: 0.0104
	LDA: 0.0888
	Avg: 0.0372
Evaluating for k=12
	PCA: 0.0106
	SVD: 0.0117
	LDA: 0.0981
	Avg: 0.0401
Evaluating for k=13
	PCA: 0.0107
	SVD: 0.0125
	LDA: 0.0995
	Avg: 0.0409
Evaluating for k=14
	PCA: 0.0111
	SVD: 0.0110
	LDA: 0.1036
	Avg: 0.0419
Evaluating for k=15
	PCA: 0.0115
	SVD: 0.0107
	LDA: 0.1100
	Avg: 0.0441
Evaluating for k=16
	PCA: 0.0114
	SVD: 0.0110
	LDA: 0.0953
	Avg: 0.0392
Evaluating for k=17
	PCA: 0.0126
	SVD: 0.0107
	LDA: 0.0844
	Avg: 0.0359
Evaluating for k=18
	PCA: 0.0132
	SVD: 0.0109
	LDA: 0.0792
	Avg: 0.0344

Optimal number of clusters for K-means based on average silhouette score: 15
